# CIA Factbook
We're going to be using a version of the CIA World Factbook (Factbook) database from the guided project from the SQL Fundamentals course. This database had one table called facts, where each row represented a country from the Factbook. 

### Facts table
Here are the first 5 rows of the facts table:

In [10]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("factbook.db")
cursor = conn.cursor()

In [15]:
# first 5 rows of fact table
q = """
SELECT *
FROM facts
LIMIT 5
"""
pd.read_sql_query(q, conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Cities table

We want to look at a new table, called cities which contains information on major urban areas from countries in the Factbook (for the rest of this mission, we'll use the word 'cities' to mean the same as 'major urban areas').

Here are the first few rows of this new table and a description of what each column represents:

In [16]:
q = """
SELECT * 
FROM cities
LIMIT 5;
"""

pd.read_sql_query(q, conn)

,id,name,population,capital,facts_id
0,1,Oranjestad,37000,1,216
1,2,Saint John'S,27000,1,6
2,3,Abu Dhabi,942000,1,184
3,4,Dubai,1978000,0,184
4,5,Sharjah,983000,0,184


Column metadata:
- id - A unique ID for each city.
- name - The name of the city.
- population - The population of the city.
- capital - Whether the city is a capital city: 1 if it is, 0 if it isn't.
- facts_id - The ID of the country, from the facts table.

Now we can join both tables together with the id column from facts and the facts_id from cities.

### Inner Join the tables

In [12]:
q = """
SELECT *
FROM facts
INNER JOIN cities
ON facts.id = cities.facts_id
LIMIT 10;
"""
pd.read_sql_query(q, conn)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,id,name,population,capital,facts_id
0,216,aa,Aruba,180,180,0,112162,1.33,12.56,8.18,8.92,1,Oranjestad,37000,1,216
1,6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21,2,Saint John'S,27000,1,6
2,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,3,Abu Dhabi,942000,1,184
3,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,4,Dubai,1978000,0,184
4,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,5,Sharjah,983000,0,184
5,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51,6,Kabul,3097000,1,1
6,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92,7,Algiers,2916000,1,3
7,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92,8,Oran,783000,0,3
8,11,aj,Azerbaijan,86600,82629,3971,9780780,0.96,16.64,7.07,0.00,9,Baku,2123000,1,11
9,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30,10,Tirana,419000,1,2


The inner join will include:

- Rows from the cities table that have a cities.facts_id that matches a facts.id from facts.

The inner join will not include:

- Rows from the cities table that have a cities.facts_id that doesn't match any facts.id from facts.
- Rows from the facts table that have a facts.id that doesn't match any cities.facts_id from cities.

### Using alias to rename the tables:

In [17]:
q = """
SELECT c.*, f.name AS country_name
FROM facts AS f
INNER JOIN cities AS c
ON f.id = c.facts_id
LIMIT 5;
"""

pd.read_sql_query(q, conn)

,id,name,population,capital,facts_id,country_name
0,1,Oranjestad,37000,1,216,Aruba
1,2,Saint John'S,27000,1,6,Antigua and Barbuda
2,3,Abu Dhabi,942000,1,184,United Arab Emirates
3,4,Dubai,1978000,0,184,United Arab Emirates
4,5,Sharjah,983000,0,184,United Arab Emirates


Next we want to produce a table of countries and their capital cities from our database. Not all countries have capital cities so I will use a WHERE clause.

### Capital of countries
We're only interested in the capital cities from the cities table.

In [18]:
q = """
SELECT f.name AS country, c.name as capital_city
FROM cities AS c
INNER JOIN facts AS f
ON f.id = c.facts_id
WHERE c.capital = 1;
"""

pd.read_sql_query(q, conn)

,country,capital_city
0,Aruba,Oranjestad
1,Antigua and Barbuda,Saint John'S
2,United Arab Emirates,Abu Dhabi
3,Afghanistan,Kabul
4,Algeria,Algiers
...,...,...
203,Samoa,Apia
204,Swaziland,Mbabane
205,Yemen,Sanaa
206,Zambia,Lusaka


### Explore missing data and total values

In [20]:
q = """
SELECT COUNT(DISTINCT(name))
FROM facts;
"""
pd.read_sql_query(q,conn)

,COUNT(DISTINCT(name))
0,261


In [22]:
q = """
SELECT COUNT(DISTINCT(facts_id))
FROM cities;
"""
pd.read_sql_query(q,conn)

,COUNT(DISTINCT(facts_id))
0,210


By running these two queries, we can see that there are some countries in the facts table that don't have corresponding cities in the cities table, which indicates we may have some incomplete data.

Next find countries from facts that don't have a corresponding value in cities.

### Using Inner Joins and Where to find NULLS

In [24]:
q="""SELECT f.name AS country, f.population
FROM facts AS f
LEFT JOIN cities AS c
ON f.id = c.facts_id
WHERE c.name IS NULL
LIMIT 5;
"""

pd.read_sql_query(q, conn)

,country,population
0,Kosovo,1870981
1,Monaco,30535
2,Nauru,9540
3,San Marino,33020
4,Singapore,5674472


The countries above in the last query do not have any city names.

Looking through the results of the query we wrote in the previous screen, we can see a number of different reasons that countries don't have corresponding values in cities:

- Countries with small populations and/or no major urban areas (which are defined as having populations of over 750,000), eg San Marino, Kosovo, and Nauru.
- City-states, such as Monaco and Singapore.
- Territories that are not themselves countries, such as Hong Kong, Gibraltar, and the Cook Islands.
- Regions & Oceans that aren't countries, such as the European Union and the Pacific Ocean.
- Genuine cases of missing data, such as Taiwan.

### Find high populated cities

In [25]:
q = """
SELECT c.name as capital_city, f.name as country, c.population
FROM facts AS f
INNER JOIN cities AS c
ON f.id = c.facts_id
WHERE c.capital = 1
ORDER BY c.population DESC
LIMIT 10;
"""

pd.read_sql_query(q,conn)

,capital_city,country,population
0,Tokyo,Japan,37217000
1,New Delhi,India,22654000
2,Mexico City,Mexico,20446000
3,Beijing,China,15594000
4,Dhaka,Bangladesh,15391000
5,Buenos Aires,Argentina,13528000
6,Manila,Philippines,11862000
7,Moscow,Russia,11621000
8,Cairo,Egypt,11169000
9,Jakarta,Indonesia,9769000


### Subquerying

Query to find the capital cities with populations of over 10 million.

In [26]:
q = """
SELECT c.name AS capital_city, f.name AS country, c.population
FROM facts AS f
INNER JOIN (
            SELECT *
            FROM cities
            WHERE capital = 1 AND population > 10000000) c
ON f.id = c.facts_id
ORDER BY c.population DESC;
"""

pd.read_sql_query(q, conn)

,capital_city,country,population
0,Tokyo,Japan,37217000
1,New Delhi,India,22654000
2,Mexico City,Mexico,20446000
3,Beijing,China,15594000
4,Dhaka,Bangladesh,15391000
5,Buenos Aires,Argentina,13528000
6,Manila,Philippines,11862000
7,Moscow,Russia,11621000
8,Cairo,Egypt,11169000


### Complex subquery
Query to find the countries where the urban center (city) population is more than half of the country's total population.

the query will include:

- A join to a subquery.
- A subquery to make a calculation.
- An aggregate function.
- A WHERE clause.
- A CAST expression.

In [27]:
q = """
SELECT 
    f.name AS country,
    c.urban_pop,
    f.population AS total_pop,
    (c.urban_pop/CAST(f.population AS FLOAT)) AS urban_pct
FROM facts AS f
INNER JOIN (
    SELECT  facts_id,
            SUM(population) AS urban_pop
    FROM cities
    GROUP BY 1) c
ON f.id = c.facts_id
WHERE urban_pct > 0.5
ORDER BY urban_pct ASC;
"""

pd.read_sql_query(q, conn)

,country,urban_pop,total_pop,urban_pct
0,Uruguay,1672000,3341893,0.500315
1,"Congo, Republic of the",2445000,4755097,0.514185
2,Brunei,241000,429646,0.560927
3,New Caledonia,157000,271615,0.578024
4,Virgin Islands,60000,103574,0.579296
5,Falkland Islands (Islas Malvinas),2000,3361,0.595061
6,Djibouti,496000,828324,0.598800
7,Australia,13789000,22751014,0.606083
8,Iceland,206000,331918,0.620635
9,Israel,5226000,8049314,0.649248


__New Columns:__
- urban_pop, the sum of the population in major urban areas belonging to that country.
- total_pop, the total population of the country.
- urban_pct, the percentage of the population within urban areas, calculated by dividing urban_pop by total_pop.